In [1]:
from datetime import datetime, timedelta
import pandas as pd
from pathlib import Path
import openpyxl

In [2]:
# Define the file path
# file_path = Path('C:/Users/falsafh0/AppData/Roaming/OpenText/OTEdit/EC_nexus/c156407569/AMSI 6 Weekly Planning (156412017).xlsm')
file_path = Path('../resource/AMSI 6 Weekly Planning (156412017).xlsm')

In [3]:
# Define the range
sheet_name = '6 Weekly'
header_row = 8
start_row = 9  # Data starts from row 9
end_row = 1485
start_col = 'I'
end_col = 'EL'

In [4]:
FieldTeam = ['Dams','Projects','CATCHMENT & ENVIRONMENT','CATCHMENT & ENVIRONMENT-Bunbury','Workshop','Conveyance']

In [5]:
# Define the column range
column_range = f"{start_col}:{end_col}"

In [6]:
# Read the data
data = pd.read_excel(
    file_path,
    sheet_name=sheet_name,
    header=header_row,  # Row 8 is header, zero-indexed as 16
    usecols=column_range,  # Use the column range directly
    skiprows=header_row,  # Skip rows up to the header row
    nrows=end_row-start_row+1
)

In [7]:
# Find today's date
today = datetime.today()
current_month = today.month
current_year = datetime.today().year

In [8]:
# Initialize an empty list to store weekly plans
weekly_plans = []

# Loop through the next 5 Mondays
for i in range(7):
    # Calculate the date of the Monday i weeks later
    monday = today + timedelta(weeks=i, days=-today.weekday())
    monday = monday.date()
    
    # Find the index in data.columns that is equal to monday
    index_of_monday = None
    for idx, column in enumerate(data.columns):
        if isinstance(column, datetime) and column.date() == monday:
            index_of_monday = idx
            break
    
    if index_of_monday is not None:
        # Create the pivot table for the current Monday
        pivot_df = data.pivot_table(
            index=[data.columns[index_of_monday], 'Region.1', 'Work order','Proj Mgr.','Project Title.1', 'Task -List.1', 'Main Field Team\n/\nTeam.1'],
            values='Total Task hours.1',
            aggfunc='sum'
        )
        
        # Filter pivot_df based on a condition (either 'x' or 'X' in the index)
        filtered_pivot_df = pivot_df[
            (pivot_df.index.get_level_values(data.columns[index_of_monday]) == 'x') |
            (pivot_df.index.get_level_values(data.columns[index_of_monday]) == 'X')
        ]
        
        # Drop the data.columns[index_of_monday] column from filtered_pivot_df
        filtered_pivot_df = (
            filtered_pivot_df
            .reset_index()
            .drop(columns=[data.columns[index_of_monday]])
            .rename(columns={
                'Region.1': 'Region',
                'Project Title.1': 'Project Title',
                'Task -List.1': 'Task',
                'Main Field Team\n/\nTeam.1': 'Team',
                 'Total Task hours.1': 'Total hours'
            })
            .set_index(['Work order','Region','Proj Mgr.','Project Title', 'Task', 'Team'])
        )

        # # Add the 'Date' column with the Monday date
        # filtered_pivot_df['Date'] = monday
        
        # Append the DataFrame to the list of weekly plans DataFrames
        weekly_plans.append(filtered_pivot_df)
        
# Initialize a dictionary to store filtered DataFrames for each FieldTeam
team_weekly_plans = {team: [] for team in FieldTeam}
# Filter each weekly plan DataFrame based on FieldTeam
for team in FieldTeam:
    for weekly_plan in weekly_plans:
        filtered_plan = weekly_plan[weekly_plan.index.get_level_values('Team') == team]
        team_weekly_plans[team].append(filtered_plan)

In [9]:
team_weekly_plans['Dams']

[                                                                                       Total hours
 Work order Region      Proj Mgr. Project Title       Task                        Team             
 0          AMSI        0         Faults              Standby day                 Dams         10.0
 981795     Metro Water Andrew H  Churchman Brook Dam Dam Maintenance Visit (6M)  Dams         13.0
                                  Serpentine Dam      Trip Prep                   Dams          1.0
 981815     SWR         Andrew H  Mungalup Dam        Dam Maintenance Visit (12M) Dams          9.5
                                  Waroona Dam         Dam Maintenance Visit (6M)  Dams         36.0,
                                                                                                                     Total hours
 Work order Region      Proj Mgr. Project Title                       Task                                     Team             
 568714     Metro Water Andrew H  Serpent

In [26]:
# Initialize an empty list to store monthly plans DataFrames
monthly_plans = []

# Find the indices of columns that have the same month as the current date
for i in range(2):
    # Find the indices of columns that have the same month and year as the current date
    index_of_months = []
    for idx, column in enumerate(data.columns):
        if isinstance(column, datetime) and column.month == current_month+i and column.year == current_year:
            index_of_months.append(idx)
            
    # Initialize an empty list to store monthly plans DataFrames for the current iteration
    monthly_plans_dfs = []

    # Loop through the indices and create pivot tables
    for index_of_month in index_of_months:
        # Extract column name for the current month
        month_column_name = data.columns[index_of_month]

        # Create the pivot table for the current month column
        pivot_df = data.pivot_table(
            index=[month_column_name, 'Region.1', 'Work order', 'Project Title.1', 'Task -List.1', 'Main Field Team\n/\nTeam.1', 'Proj Mgr.'],
            values='Total Task hours.1',
            aggfunc='sum'
        )

        # Filter pivot_df based on a condition ('x' or 'X' in the index)
        filtered_pivot_df = pivot_df[
            (pivot_df.index.get_level_values(month_column_name) == 'x') |
            (pivot_df.index.get_level_values(month_column_name) == 'X')
        ]

        # Rename columns and set index
        filtered_pivot_df = (
            filtered_pivot_df
            .reset_index()
            .drop(columns=[month_column_name])
            .rename(columns={
                'Region.1': 'Region',
                'Project Title.1': 'Project Title',
                'Task -List.1': 'Task',
                'Main Field Team\n/\nTeam.1': 'Team',
                'Proj Mgr.': 'Proj Mgr.',
                'Total Task hours.1': 'Total hours'
            })
            .set_index(['Region', 'Work order', 'Proj Mgr.', 'Project Title', 'Task', 'Team'])
        )

        # Append the filtered DataFrame to the list of monthly plans DataFrames for the current iteration
        monthly_plans_dfs.append(filtered_pivot_df)

    # Concatenate all monthly plans DataFrames into a single DataFrame for this month
    concatenated_df = pd.concat(monthly_plans_dfs)

    # Group by 'Work order' and sum the 'Total hours'
    grouped_df = concatenated_df.groupby(['Work order', 'Region', 'Proj Mgr.', 'Project Title', 'Task', 'Team'])['Total hours'].sum().reset_index()

    # Append the grouped DataFrame to the list of grouped monthly plans DataFrames
    monthly_plans.append(grouped_df)

# Initialize a dictionary to store filtered monthly plans for each FieldTeam
team_monthly_plans = {team: [] for team in FieldTeam}

# Filter each monthly plan DataFrame based on FieldTeam
for team in FieldTeam:
    for monthly_plan in monthly_plans:
        # Filter the DataFrame where the 'Team' index matches the current team
        filtered_plan = monthly_plan.reset_index()
        filtered_plan = filtered_plan[filtered_plan['Team'] == team]
        team_monthly_plans[team].append(filtered_plan)

In [11]:
team_monthly_plans['CATCHMENT & ENVIRONMENT'][1]

,index,Work order,Region,Proj Mgr.,Project Title,Task,Team,Total hours
2,2,568654,Metro Water,Andrew H,South Jandakot Abstraction Trial- DOWER Bore,Data Validation,CATCHMENT & ENVIRONMENT,5.0
3,3,568654,Metro Water,Andrew H,South Jandakot Abstraction Trial- DOWER Bore,RSV (total hours),CATCHMENT & ENVIRONMENT,5.0
4,4,568654,Metro Water,Andrew H,South Jandakot Abstraction Trial- DOWER Bore,Travel,CATCHMENT & ENVIRONMENT,2.0
7,7,568706,NWR,Glenn,Millstream Bore level Monitoring Project,Bore level instrument annual visit,CATCHMENT & ENVIRONMENT,27.0
8,8,568706,NWR,Glenn,Millstream Bore level Monitoring Project,Flow sensors annual Visit,CATCHMENT & ENVIRONMENT,24.0
...,...,...,...,...,...,...,...,...
261,261,983639,NWR,Glenn,Derby Bore Monitoring,Validate and Archive,CATCHMENT & ENVIRONMENT,3.0
262,262,984328,Statewide,Elise,Broome North WRRF Ground Water Monitoring,Broom Bores Water sampling,CATCHMENT & ENVIRONMENT,30.0
263,263,984328,Statewide,Elise,Broome North WRRF Ground Water Monitoring,Travel-1st visit,CATCHMENT & ENVIRONMENT,20.0
264,264,984328,Statewide,Elise,Broome North WRRF Ground Water Monitoring,Travel-routin visits,CATCHMENT & ENVIRONMENT,10.0


In [18]:
monthly_plans[0]

,Work order,Region,Proj Mgr.,Project Title,Task,Team,Total hours
0,0,AMSI,0,Faults,Standby day,Dams,20.0
1,568654,Metro Water,Andrew H,South Jandakot Abstraction Trial- DOWER Bore,Data Reporting,Projects,1.0
2,568654,Metro Water,Andrew H,South Jandakot Abstraction Trial- DOWER Bore,Data Validation,CATCHMENT & ENVIRONMENT,5.0
3,568654,Metro Water,Andrew H,South Jandakot Abstraction Trial- DOWER Bore,RSV (total hours),CATCHMENT & ENVIRONMENT,5.0
4,568654,Metro Water,Andrew H,South Jandakot Abstraction Trial- DOWER Bore,Travel,CATCHMENT & ENVIRONMENT,2.0
...,...,...,...,...,...,...,...
256,F-A9999-xxxx,The Inactive,Glenn,BINDOON PRV Noise Monitoring,Data Handover,Projects,1.5
257,F-A9999-xxxx,The Inactive,Glenn,BINDOON PRV Noise Monitoring,Investigation,Conveyance,1.0
258,F-A9999-xxxx,The Inactive,Glenn,BINDOON PRV Noise Monitoring,Travel,Conveyance,4.0
259,F-A9999-xxxx,The Inactive,Glenn,BINDOON PRV Noise Monitoring,Trip Prep,Conveyance,2.0


In [28]:
# # Define the output Excel file path
outexcel_file_path = Path('../resource/finaldata.xlsx')

# Save the pivot table to an Excel file
team_monthly_plans['CATCHMENT & ENVIRONMENT'][1].to_excel(outexcel_file_path, index=False)

In [27]:
index_of_months

[9, 10, 11, 12]

In [25]:
current_month+1

8